# Air quality : Transitive frontier learnig Example

Below is an example to implement a Transitive Frontier Learning on the Air quality dataset.
We first show how we processed the dataset, then how to perform the analysis.

In [19]:
import pandas as pd
import datetime as dt
from math import log2  
import pydotplus as ptp 

# helper_classes 
import import_ipynb
import helper_general as my
from ConstraintDefiner import ConstraintDefiner as Lark_CD


## Loading + preprocessing Dataset

In [20]:
# Take the correct rows and columns
airQuality = pd.read_csv("./AirQualityUCI.csv", sep=';').iloc[0:9356,:-2] 
display(airQuality.head(3))

# Replacement the "," with "." in the scientific notations 
airQuality = airQuality.astype(str).applymap(lambda val: val.replace(',','.'))

# Replacement the "(" and ")" with "_" in the columns' name to prevent grammar's errors
origNames = airQuality.columns.copy()
airQuality.columns = [ colName.replace("(","_").replace(')','').replace('.','_') for colName in airQuality.columns.to_list()]

# Add weekend
airQuality['Date'] = (airQuality['Date'].map(lambda x: dt.datetime.strptime(x,'%d/%m/%Y')).dt.dayofweek >= 5).astype(int)
airQuality = airQuality.rename(columns={'Date':'Weekend'})
display(airQuality.head(3))

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,10/03/2004,18.00.00,"2,6",1360.0,150.0,"11,9",1046.0,166.0,1056.0,113.0,1692.0,1268.0,"13,6","48,9","0,7578"
1,10/03/2004,19.00.00,2,1292.0,112.0,"9,4",955.0,103.0,1174.0,92.0,1559.0,972.0,"13,3","47,7","0,7255"
2,10/03/2004,20.00.00,"2,2",1402.0,88.0,"9,0",939.0,131.0,1140.0,114.0,1555.0,1074.0,"11,9","54,0","0,7502"


,Weekend,Time,CO_GT,PT08_S1_CO,NMHC_GT,C6H6_GT,PT08_S2_NMHC,NOx_GT,PT08_S3_NOx,NO2_GT,PT08_S4_NO2,PT08_S5_O3,T,RH,AH
0,0,18.00.00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578
1,0,19.00.00,2,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255
2,0,20.00.00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502


## Mapping values, Encoder and Decoder
Mapping

In [21]:
# Values mapping into integer values
encod , decod = {},{}
for col in airQuality.columns.to_list():
    
    if col == "Time":
        dec = airQuality[col].drop_duplicates().sort_values().reset_index(drop=True)
    elif col == "Weekend":
        dec = airQuality[col].drop_duplicates().sort_values().reset_index(drop=True)
    else :
        dec = airQuality[col].astype(float).drop_duplicates().sort_values() #float-> -200 == -200.0 
        dec = dec.iloc[1:].reset_index(drop=True) # remove the -200.0 (None) => mapping only between not None values    
    enc = dec.reset_index().set_index(col)['index']
    
    encod |= {col: dict(enc)| {-200.0: -1} } # In orther to have only integer values
    decod |= {col: dict(dec)| {-1 : -200.0}}
#my.myDisplay([encod['CO_GT'], decod['CO_GT']], axis=1)

Encoding Dataset

In [22]:
df_AQ = pd.DataFrame() 
for col in airQuality.columns:
    if col == "Time":
         df_AQ[col] = airQuality[col].map(lambda val: encod[col][val])
    else:
        df_AQ[col] = airQuality[col].map(lambda val: encod[col][float(val)]) # cast in float -> '2' == '2.0' == 2.0
df_AQ

,Weekend,Time,CO_GT,PT08_S1_CO,NMHC_GT,C6H6_GT,PT08_S2_NMHC,NOx_GT,PT08_S3_NOx,NO2_GT,PT08_S4_NO2,PT08_S5_O3,T,RH,AH
0,0,18,25,665,135,118,622,162,699,107,985,976,145,379,1894
1,0,19,19,597,100,93,531,99,817,86,852,680,142,367,1725
2,0,20,21,707,77,89,515,127,783,108,848,782,128,430,1851
3,0,21,21,681,69,91,524,168,735,116,877,911,119,490,2054
4,0,22,15,577,40,64,412,127,847,110,783,818,121,486,2064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9351,0,9,38,602,-1,135,678,517,150,181,668,1283,191,253,1845
9352,0,10,30,619,-1,134,677,467,182,184,667,1420,228,183,1887
9353,0,11,23,468,-1,113,603,349,247,173,557,977,252,127,1656
9354,0,12,23,447,-1,123,639,289,246,169,534,800,278,73,1356


Decoding Dataset

In [23]:
df_AQ_2 = pd.DataFrame() 
for col in airQuality.columns:
         df_AQ_2[col] = df_AQ[col].map(lambda val: decod[col][val])
df_AQ_2

,Weekend,Time,CO_GT,PT08_S1_CO,NMHC_GT,C6H6_GT,PT08_S2_NMHC,NOx_GT,PT08_S3_NOx,NO2_GT,PT08_S4_NO2,PT08_S5_O3,T,RH,AH
0,0,18.00.00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578
1,0,19.00.00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255
2,0,20.00.00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502
3,0,21.00.00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867
4,0,22.00.00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9351,0,09.00.00,3.9,1297.0,-200.0,13.6,1102.0,523.0,507.0,187.0,1375.0,1583.0,18.2,36.3,0.7487
9352,0,10.00.00,3.1,1314.0,-200.0,13.5,1101.0,472.0,539.0,190.0,1374.0,1729.0,21.9,29.3,0.7568
9353,0,11.00.00,2.4,1163.0,-200.0,11.4,1027.0,353.0,604.0,179.0,1264.0,1269.0,24.3,23.7,0.7119
9354,0,12.00.00,2.4,1142.0,-200.0,12.4,1063.0,293.0,603.0,175.0,1241.0,1092.0,26.9,18.3,0.6406


## Decode Air Quality Rule

In [24]:
def AQ_decodeRule(decod, df_X, df_Y):
    def colDec(col, name):
        return col.map(lambda v: decod[name][v])  
    
    X = df_X.T.apply(lambda col: colDec(col, col.name)).T
    Y = df_Y.T.apply(lambda col: colDec(col, col.name)).T
    
    #--------------------------
    # Pretty rule
    X_pretty = X.index+'['+ X.L.astype(str)+', '+X.U.astype(str)+']'
    Y_pretty = Y.index+'['+ Y.L.astype(str)+', '+Y.U.astype(str)+']'
    s = f"{', '.join(X_pretty.tolist())} --> {', '.join(Y_pretty.tolist())}"
    
    return X,Y,s

# dX,dY,ds = AQ_decodeRule(decod, X,Y) 
# my.myDisplay([dX,dY], axis=1)
# ds

## Support and Confidence

In [25]:
def supp_conf(df_Coded, df_X, df_Y, tempShow=False):
    df_temp = pd.DataFrame()
    for col in df_X.index:
        df_temp[col] = (df_X['L'][col] <= df_Coded[col]) & (df_Coded[col] <= df_X['U'][col])
    
    for col in df_Y.index:
        df_temp[col] = (df_Y['L'][col] <= df_Coded[col]) & (df_AQ[col] <= df_Y['U'][col])
    
    #-----------------------------------------
    # Calcolo freq X e XY
    freqX = df_temp[ df_temp[df_X.index].T.sum() == len(df_X)] # freq of X 
    freqXY = freqX[freqX[df_Y.index].T.sum() == len(df_Y)]     # freq of XY
    
    if tempShow : my.myDisplay([df_temp,freqX,freqXY],names=['Evaluation:','X:','XY:'], axis=1)
    return {'supp':len(freqXY)/len(df_Coded) , 'conf':0 if len(freqX)==0 else len(freqXY)/len(freqX)}

# X,Y,s = c.decodeRule(res)
# suppConf = supp_conf(df_AQ, X, Y, True)
# suppConf

## Test Pipeline 

In [26]:
# after preprocessing
df_AQ.head(3)

,Weekend,Time,CO_GT,PT08_S1_CO,NMHC_GT,C6H6_GT,PT08_S2_NMHC,NOx_GT,PT08_S3_NOx,NO2_GT,PT08_S4_NO2,PT08_S5_O3,T,RH,AH
0,0,18,25,665,135,118,622,162,699,107,985,976,145,379,1894
1,0,19,19,597,100,93,531,99,817,86,852,680,142,367,1725
2,0,20,21,707,77,89,515,127,783,108,848,782,128,430,1851


In [27]:
# Select Vars to perform the analysis
df_var = df_AQ.max().to_frame().reset_index()#.iloc[[1,12,13,14],:]
df_var

,index,0
0,Weekend,1
1,Time,23
2,CO_GT,95
3,PT08_S1_CO,1040
4,NMHC_GT,428
5,C6H6_GT,406
6,PT08_S2_NMHC,1244
7,NOx_GT,924
8,PT08_S3_NOx,1220
9,NO2_GT,282


###  ConstraintDefiner Initialization

This class is designed to create an interface between the user and the SAT-like core. 
It takes the variables we are interested on and their maximum value to generate constraints to guide the SAT-like search.

This class try to find ranges where a rule could have some meanig. For each variable is defined a lower bond and an upper bound. The value of each variable (and its lower and upper bound) and their is represented in bits, and its evaluation is done bit by bit.

The constraints implemented are: 

    init_constr(): that describes the structure of the rule that must be respected and the values' interval (to zero from ther maximum  values 
    NotSupported(res): Describes the constraint we want to apply in the next search if the obtained rule was not supported. 
    NotConfident(res): Describes the constraint we want to apply in the next search if the obtained rule was not confident.
    Confident(res):    Describes the constraint we want to apply in the next search if the obtained rule was confident.
    reject_solut(res): Describes the constraint we want to apply in the next search if we don't want the same rule again.


In [42]:
c = Lark_CD(df_var['index'], df_var[0]) 
c.addConstr( c.init_constr() )  
c.set_resetTree(False) 

In [43]:
c.set_seed(2981050372)  # we can set a seed to reproducibility
c.set_childrenChoice('min_power') #'random' or 'min_avgLenPath' or 'min_power' (soft_max)
c.set_priority(digitPrior='Complete') # none, 'Simple', 'Complete'
c.set_saturate_freeVars(saturate='effective') # None, 'all', 'effective'
fileName = 'SCE'

#### Start search

In [ ]:
# %%capture --no-stderr output

# Decide support and confidence
supp = 0.05
conf = 0.8
duration = '10h' # test durations (s:seconds, m:minutes, h:hours)

#----------------------
i, allBool = 0, False
sol, state_Sol, data_Solut= [], [], []

#--------------------
duration = pd.Timedelta(duration)
startTime = dt.datetime.now() # only for test

#---------------------------
while (dt.datetime.now() - startTime ) < duration:  
    #print(dt.datetime.now() - startTime)
    start_run = dt.datetime.now()
    nChar = sum([len(constr) for constr in c.constr])
    print(f"Run:{i+1}, #char:{nChar}, Started:{start_run.strftime('%m-%d %H:%M:%S')}, ", end='')
    print("")
    #-----------------------
    # SAT
    res = c.Sat_Solver(test='speed') # only for test, else:  c.Sat_Solver()
    end_run = dt.datetime.now()
    
    if res is None: break    # noSoluzione
    if len(res)==0:  
        allBool=True    # anly true and false -> ris={}
        break
       
    #-----------------------
    # Supp + Conf
    X,Y,s = c.decodeRule(res) # X -> antecedent, Y-> consequent, L->lower bount, U->upper bound
    suppConf = supp_conf(df_AQ, X, Y)
    
    sol = {'solut':s } | suppConf
    if suppConf['supp'] < supp:
        c.addConstr( c.NotSupported(res) )
        sol |= {'state':'Not supported'}
    
    elif suppConf['conf'] < conf:
        c.addConstr( c.NotConfident(res) )
        sol |= {'state':'Not confident'}
    
    else:
        c.addConstr( c.Confident(res) )
        sol |= {'state':'Confidend'}
    
    c.addConstr(  c.reject_solut(res) )
    print(f"runTime:{str(end_run - start_run)[:-7]}, TotTime:{str(end_run-startTime)[:-7]}, State:{sol['state']},")
 
    #-----------------------------
    # Save sol
    state_Sol.append(sol) 
    
    X[['Side','idRule']], Y[['Side','idRule']] = ('X',i),('Y',i)
    data_Solut.append([X,Y])
    i = i+1
    

Run:1, #char:386321, Started:11-30 14:00:05, 
-->, SunRun:1, Started:11-30 14:00:05, runTime:0:00:01.019, 
-->, SunRun:2, Started:11-30 14:00:06, runTime:0:00:01.274, 
-->, SunRun:3, Started:11-30 14:00:07, runTime:0:00:00.746, 
-->, SunRun:4, Started:11-30 14:00:08, runTime:0:00:01.071, 
-->, SunRun:5, Started:11-30 14:00:09, runTime:0:00:00.997, 
runTime:0:00:05, TotTime:0:00:05, State:Not supported,
Run:2, #char:415082, Started:11-30 14:00:10, 
-->, SunRun:1, Started:11-30 14:00:10, runTime:0:00:00.963, 
-->, SunRun:2, Started:11-30 14:00:11, runTime:0:00:01.445, 
runTime:0:00:02, TotTime:0:00:07, State:Not confident,
Run:3, #char:453179, Started:11-30 14:00:14, 
-->, SunRun:1, Started:11-30 14:00:14, runTime:0:00:02.172, 
-->, SunRun:2, Started:11-30 14:00:16, runTime:0:00:00.442, 
-->, SunRun:3, Started:11-30 14:00:16, runTime:0:00:00.727, 
-->, SunRun:4, Started:11-30 14:00:17, runTime:0:00:00.082, 
-->, SunRun:5, Started:11-30 14:00:17, runTime:0:00:00.064, 
-->, SunRun:6, Start

-->, SunRun:1, Started:11-30 14:01:47, runTime:0:00:00.901, 
-->, SunRun:2, Started:11-30 14:01:48, runTime:0:00:00.104, 
-->, SunRun:3, Started:11-30 14:01:48, runTime:0:00:00.144, 
-->, SunRun:4, Started:11-30 14:01:48, runTime:0:00:03.355, 
-->, SunRun:5, Started:11-30 14:01:51, runTime:0:00:00.483, 
-->, SunRun:6, Started:11-30 14:01:52, runTime:0:00:02.305, 
-->, SunRun:7, Started:11-30 14:01:54, runTime:0:00:00.814, 
-->, SunRun:8, Started:11-30 14:01:55, runTime:0:00:01.391, 
-->, SunRun:9, Started:11-30 14:01:56, runTime:0:00:02.372, 
-->, SunRun:10, Started:11-30 14:01:59, runTime:0:00:01.225, 
runTime:0:00:13, TotTime:0:01:54, State:Not supported,
Run:25, #char:1191190, Started:11-30 14:02:00, 
-->, SunRun:1, Started:11-30 14:02:00, runTime:0:00:02.644, 
-->, SunRun:2, Started:11-30 14:02:03, runTime:0:00:00.163, 
-->, SunRun:3, Started:11-30 14:02:03, runTime:0:00:01.875, 
runTime:0:00:04, TotTime:0:01:59, State:Not supported,
Run:26, #char:1221342, Started:11-30 14:02:05, 


runTime:0:00:01, TotTime:0:04:04, State:Not confident,
Run:49, #char:1938098, Started:11-30 14:04:10, 
-->, SunRun:1, Started:11-30 14:04:10, runTime:0:00:01.621, 
runTime:0:00:01, TotTime:0:04:06, State:Not confident,
Run:50, #char:1978113, Started:11-30 14:04:12, 
-->, SunRun:1, Started:11-30 14:04:12, runTime:0:00:00.689, 
-->, SunRun:2, Started:11-30 14:04:13, runTime:0:00:00.743, 
-->, SunRun:3, Started:11-30 14:04:14, runTime:0:00:00.255, 
-->, SunRun:4, Started:11-30 14:04:14, runTime:0:00:03.258, 
-->, SunRun:5, Started:11-30 14:04:17, runTime:0:00:01.254, 
-->, SunRun:6, Started:11-30 14:04:18, runTime:0:00:02.592, 
-->, SunRun:7, Started:11-30 14:04:21, runTime:0:00:02.325, 
runTime:0:00:11, TotTime:0:04:18, State:Not supported,
Run:51, #char:2014616, Started:11-30 14:04:24, 
-->, SunRun:1, Started:11-30 14:04:24, runTime:0:00:03.932, 
-->, SunRun:2, Started:11-30 14:04:28, runTime:0:00:03.381, 
-->, SunRun:3, Started:11-30 14:04:31, runTime:0:00:00.701, 
-->, SunRun:4, Start

-->, SunRun:4, Started:11-30 14:10:27, runTime:0:00:02.149, 
-->, SunRun:5, Started:11-30 14:10:29, runTime:0:00:00.415, 
-->, SunRun:6, Started:11-30 14:10:30, runTime:0:00:02.939, 
-->, SunRun:7, Started:11-30 14:10:33, runTime:0:00:01.143, 
-->, SunRun:8, Started:11-30 14:10:34, runTime:0:00:01.799, 
runTime:0:00:12, TotTime:0:10:30, State:Not supported,
Run:92, #char:3398609, Started:11-30 14:10:36, 
-->, SunRun:1, Started:11-30 14:10:36, runTime:0:00:00.412, 
-->, SunRun:2, Started:11-30 14:10:37, runTime:0:00:02.095, 
-->, SunRun:3, Started:11-30 14:10:39, runTime:0:00:00.866, 
-->, SunRun:4, Started:11-30 14:10:39, runTime:0:00:02.017, 
-->, SunRun:5, Started:11-30 14:10:41, runTime:0:00:01.240, 
-->, SunRun:6, Started:11-30 14:10:43, runTime:0:00:01.581, 
-->, SunRun:7, Started:11-30 14:10:44, runTime:0:00:00.324, 
-->, SunRun:8, Started:11-30 14:10:45, runTime:0:00:00.721, 
-->, SunRun:9, Started:11-30 14:10:45, runTime:0:00:02.740, 
-->, SunRun:10, Started:11-30 14:10:48, run

-->, SunRun:1, Started:11-30 14:14:17, runTime:0:00:01.352, 
-->, SunRun:2, Started:11-30 14:14:18, runTime:0:00:01.465, 
-->, SunRun:3, Started:11-30 14:14:20, runTime:0:00:04.807, 
-->, SunRun:4, Started:11-30 14:14:25, runTime:0:00:00.868, 
-->, SunRun:5, Started:11-30 14:14:26, runTime:0:00:01.818, 
-->, SunRun:6, Started:11-30 14:14:27, runTime:0:00:01.823, 
runTime:0:00:12, TotTime:0:14:24, State:Not supported,
Run:108, #char:3963725, Started:11-30 14:14:30, 
-->, SunRun:1, Started:11-30 14:14:30, runTime:0:00:02.988, 
-->, SunRun:2, Started:11-30 14:14:33, runTime:0:00:04.115, 
-->, SunRun:3, Started:11-30 14:14:37, runTime:0:00:02.344, 
runTime:0:00:09, TotTime:0:14:33, State:Not supported,
Run:109, #char:3997972, Started:11-30 14:14:39, 
-->, SunRun:1, Started:11-30 14:14:39, runTime:0:00:02.023, 
runTime:0:00:02, TotTime:0:14:36, State:Confidend,
Run:110, #char:4033614, Started:11-30 14:14:42, 
-->, SunRun:1, Started:11-30 14:14:42, runTime:0:00:02.973, 
runTime:0:00:02, TotT

...

-->, SunRun:14, Started:11-30 19:38:33, runTime:0:00:02.645, 
-->, SunRun:15, Started:11-30 19:38:36, runTime:0:00:08.301, 
-->, SunRun:16, Started:11-30 19:38:44, runTime:0:00:03.111, 
-->, SunRun:17, Started:11-30 19:38:47, runTime:0:00:11.499, 
-->, SunRun:18, Started:11-30 19:38:59, runTime:0:00:03.296, 
runTime:0:01:50, TotTime:5:38:57, State:Not supported,
Run:657, #char:22303739, Started:11-30 19:39:02, 
-->, SunRun:1, Started:11-30 19:39:02, runTime:0:00:11.840, 
-->, SunRun:2, Started:11-30 19:39:14, runTime:0:00:14.163, 
-->, SunRun:3, Started:11-30 19:39:28, runTime:0:00:04.379, 
-->, SunRun:4, Started:11-30 19:39:33, runTime:0:00:08.830, 
-->, SunRun:5, Started:11-30 19:39:42, runTime:0:00:03.891, 
runTime:0:00:43, TotTime:5:39:40, State:Not confident,
Run:658, #char:22347542, Started:11-30 19:39:46, 
-->, SunRun:1, Started:11-30 19:39:46, runTime:0:00:14.466, 
-->, SunRun:2, Started:11-30 19:40:01, runTime:0:00:07.695, 
-->, SunRun:3, Started:11-30 19:40:08, runTime:0:00:0

-->, SunRun:3, Started:11-30 20:06:53, runTime:0:00:12.270, 
-->, SunRun:4, Started:11-30 20:07:05, runTime:0:00:04.125, 
-->, SunRun:5, Started:11-30 20:07:10, runTime:0:00:02.505, 
-->, SunRun:6, Started:11-30 20:07:12, runTime:0:00:13.393, 
-->, SunRun:7, Started:11-30 20:07:25, runTime:0:00:14.505, 
-->, SunRun:8, Started:11-30 20:07:40, runTime:0:00:08.758, 
-->, SunRun:9, Started:11-30 20:07:49, runTime:0:00:02.505, 
-->, SunRun:10, Started:11-30 20:07:51, runTime:0:00:11.273, 
-->, SunRun:11, Started:11-30 20:08:03, runTime:0:00:03.644, 
-->, SunRun:12, Started:11-30 20:08:06, runTime:0:00:03.029, 
-->, SunRun:13, Started:11-30 20:08:09, runTime:0:00:10.219, 
-->, SunRun:14, Started:11-30 20:08:19, runTime:0:00:13.372, 
-->, SunRun:15, Started:11-30 20:08:33, runTime:0:00:03.867, 
-->, SunRun:16, Started:11-30 20:08:37, runTime:0:00:04.115, 
runTime:0:02:04, TotTime:6:08:35, State:Not supported,
Run:688, #char:23345733, Started:11-30 20:08:41, 
-->, SunRun:1, Started:11-30 20:08

-->, SunRun:1, Started:11-30 20:34:12, runTime:0:00:02.362, 
-->, SunRun:2, Started:11-30 20:34:14, runTime:0:00:11.203, 
-->, SunRun:3, Started:11-30 20:34:26, runTime:0:00:03.311, 
-->, SunRun:4, Started:11-30 20:34:29, runTime:0:00:02.552, 
-->, SunRun:5, Started:11-30 20:34:32, runTime:0:00:02.529, 
-->, SunRun:6, Started:11-30 20:34:34, runTime:0:00:03.241, 
runTime:0:00:25, TotTime:6:34:32, State:Confidend,
Run:721, #char:24526436, Started:11-30 20:34:38, 
-->, SunRun:1, Started:11-30 20:34:38, runTime:0:00:09.217, 
-->, SunRun:2, Started:11-30 20:34:47, runTime:0:00:02.844, 
-->, SunRun:3, Started:11-30 20:34:50, runTime:0:00:03.628, 
runTime:0:00:15, TotTime:6:34:48, State:Not confident,
Run:722, #char:24565549, Started:11-30 20:34:54, 
-->, SunRun:1, Started:11-30 20:34:54, runTime:0:00:03.875, 
-->, SunRun:2, Started:11-30 20:34:58, runTime:0:00:02.742, 
-->, SunRun:3, Started:11-30 20:35:01, runTime:0:00:07.944, 
-->, SunRun:4, Started:11-30 20:35:09, runTime:0:00:05.716, 
-

-->, SunRun:5, Started:11-30 21:06:21, runTime:0:00:03.826, 
-->, SunRun:6, Started:11-30 21:06:24, runTime:0:00:13.047, 
-->, SunRun:7, Started:11-30 21:06:37, runTime:0:00:02.693, 
-->, SunRun:8, Started:11-30 21:06:40, runTime:0:00:09.050, 
-->, SunRun:9, Started:11-30 21:06:49, runTime:0:00:14.876, 
-->, SunRun:10, Started:11-30 21:07:04, runTime:0:00:05.522, 
-->, SunRun:11, Started:11-30 21:07:10, runTime:0:00:26.769, 
-->, SunRun:12, Started:11-30 21:07:36, runTime:0:00:14.302, 
-->, SunRun:13, Started:11-30 21:07:51, runTime:0:00:13.230, 
-->, SunRun:14, Started:11-30 21:08:04, runTime:0:00:21.656, 
-->, SunRun:15, Started:11-30 21:08:26, runTime:0:00:04.040, 
runTime:0:02:31, TotTime:7:08:24, State:Not supported,
Run:750, #char:25530408, Started:11-30 21:08:30, 
-->, SunRun:1, Started:11-30 21:08:30, runTime:0:00:14.630, 
-->, SunRun:2, Started:11-30 21:08:45, runTime:0:00:03.077, 
-->, SunRun:3, Started:11-30 21:08:48, runTime:0:00:10.791, 
-->, SunRun:4, Started:11-30 21:08:

-->, SunRun:5, Started:11-30 21:39:51, runTime:0:00:16.539, 
-->, SunRun:6, Started:11-30 21:40:07, runTime:0:00:02.831, 
runTime:0:00:47, TotTime:7:40:04, State:Not supported,
Run:776, #char:26414043, Started:11-30 21:40:10, 
-->, SunRun:1, Started:11-30 21:40:10, runTime:0:00:02.431, 
runTime:0:00:02, TotTime:7:40:07, State:Not supported,
Run:777, #char:26431955, Started:11-30 21:40:13, 
-->, SunRun:1, Started:11-30 21:40:13, runTime:0:00:03.543, 
-->, SunRun:2, Started:11-30 21:40:16, runTime:0:00:06.845, 
-->, SunRun:3, Started:11-30 21:40:23, runTime:0:00:03.051, 
-->, SunRun:4, Started:11-30 21:40:26, runTime:0:00:16.054, 
-->, SunRun:5, Started:11-30 21:40:42, runTime:0:00:02.794, 
runTime:0:00:32, TotTime:7:40:40, State:Not supported,
Run:778, #char:26459115, Started:11-30 21:40:45, 
-->, SunRun:1, Started:11-30 21:40:45, runTime:0:00:12.483, 
-->, SunRun:2, Started:11-30 21:40:58, runTime:0:00:25.525, 
-->, SunRun:3, Started:11-30 21:41:23, runTime:0:00:02.325, 
-->, SunRun:4,

In [ ]:
# Save constraints
#c.save_Constr_txt( 'AQ_contrsFile' )

In [45]:
# Save state of the tested rules
pd.DataFrame(state_Sol).to_csv(f'{fileName}_state.csv')
pd.DataFrame(state_Sol)

,solut,supp,conf,state
0,"PT08_S1_CO[0,16], NOx_GT[0,412] --> Time[0,7]",0.001710,0.727273,Not supported
1,"NOx_GT[0,412], RH[0,240] --> AH[4096,6681]",0.054831,0.301942,Not confident
2,"PT08_S1_CO[0,16], NOx_GT[0,412] --> PT08_S3_NO...",0.000000,0.000000,Not supported
3,"PT08_S1_CO[0,16], NOx_GT[0,412] --> C6H6_GT[25...",0.000000,0.000000,Not supported
4,"PT08_S1_CO[0,528], NOx_GT[0,412] --> Time[0,7]",0.220500,0.400738,Not confident
...,...,...,...,...
785,"NMHC_GT[0,92] --> NO2_GT[265,282], T[432,435]",0.000000,0.000000,Not supported
786,"NOx_GT[0,12] --> PT08_S1_CO[1024,1040]",0.000000,0.000000,Not supported
787,"NOx_GT[0,412] --> CO_GT[0,31]",0.580269,0.847751,Confidend
788,"PT08_S2_NMHC[0,348] --> T[256,435]",0.032172,0.111813,Not supported


In [46]:
# save the rule with range attribute
pd.concat([pd.concat(d).reset_index().set_index('idRule') for d in data_Solut]).reset_index().to_csv(f'{fileName}_solutions.csv',index=False)
pd.concat([pd.concat(d).reset_index().set_index('idRule') for d in data_Solut]).reset_index()

,idRule,V,L,U,Side
0,0,PT08_S1_CO,0,16,X
1,0,NOx_GT,0,412,X
2,0,Time,0,7,Y
3,1,NOx_GT,0,412,X
4,1,RH,0,240,X
...,...,...,...,...,...
2613,788,PT08_S2_NMHC,0,348,X
2614,788,T,256,435,Y
2615,789,PT08_S1_CO,0,144,X
2616,789,NOx_GT,0,444,X
